In [1]:
# Check platform.
import platform
if platform.machine() not in ['x86_64', 'aarch64']:
    raise SystemExit("Unsupported platform!")

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Connecting to clickhouse on remote server
import clickhouse_connect

# Import the channel estimator and some utilities for converting
# the DMRS fields in the right format from the SCF FAPI format that the dataset follows.
from aerial.phy5g.algorithms import ChannelEstimator
from aerial.util.fapi import dmrs_fapi_to_bit_array

clickhouse_client = clickhouse_connect.get_client(host='localhost')

In [2]:
import datetime
import heapq

def average_of_largest(arr, n=20):
    if len(arr) < n:
        return sum(arr) / len(arr)  # If there are less than 10 elements, take the average of all
    largest_numbers = heapq.nlargest(n, arr)  # Get the n largest numbers
    return sum(largest_numbers) / n  # Compute the average

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip3 install torch
!pip3 install torchinfo
# !pip install torch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# !pip install torchinfo
from torchinfo import summary

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
print(torch.cuda.device_count())  # Number of GPUs available
device = torch.device("cuda:0")  # Select the first GPU (index 0)
print(device)

In [8]:
import dill
model = torch.load("model_2025_04_02_19_14.pth", pickle_module=dill)

In [10]:
X_SIZE = 4
HIDDEN_SIZE = 50
NUMBER_OF_LAYER = 2
DROP_OUT = 0.1

FC_LAYER_1_SIZE = 50
FC_LAYER_2_SIZE = 25
FC_OUTPUT_SIZE = 1

WINDOWS_SIZE = 20

In [14]:
model_stats = summary(model, input_size=(1, WINDOWS_SIZE, X_SIZE))
print(model_stats)

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     [1, 1]                    --
├─LSTM: 1-1                              [1, 20, 50]               31,600
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 50]                   2,550
│    └─ReLU: 2-2                         [1, 50]                   --
│    └─Linear: 2-3                       [1, 1]                    51
Total params: 34,201
Trainable params: 34,201
Non-trainable params: 0
Total mult-adds (M): 0.63
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.14
Estimated Total Size (MB): 0.15


In [30]:
while True:
    query = f"""
    SELECT * FROM MAC_KPIs 
    ORDER BY TsTaiNs DESC
    LIMIT {WINDOWS_SIZE}
    """
    # print(query)
    kpis = clickhouse_client.query_df(query)
    # print(kpis)

    x = kpis[['phr', 'wb_cqi', 'pusch_snr', 'rsrp']].to_numpy() / np.array([100, 20, 30, -150])

    X_torch = torch.tensor([x], dtype=torch.float32)
    predicted = model(X_torch.to(device)).to("cpu").detach().numpy()
    print(f" {kpis['TsTaiNs'].iloc[-1]}: {predicted[-1][0]}")
    break

 2025-04-01 13:35:23.784000: 27.031213760375977
